In [1]:
import cpa
import scanpy as sc

[rank: 0] Global seed set to 0
2024-06-13 14:50:56,061	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-06-13 14:50:59,698	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
adata_train_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_30.h5ad"
adata_test_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_30.h5ad"
adata_ood_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_30.h5ad"

In [3]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [4]:
import anndata

adata_combined = anndata.concat((adata_train, adata_test, adata_ood), label="split", keys=["train", "test", "ood"])

In [5]:
try:
    adata = sc.read(data_path)
except:
    import gdown
    gdown.download('https://drive.google.com/uc?export=download&id=1RRV0_qYKGTvD3oCklKfoZQFYqKJy4l6t')
    data_path = 'combo_sciplex_prep_hvg_filtered.h5ad'
    adata = sc.read(data_path)

adata

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1RRV0_qYKGTvD3oCklKfoZQFYqKJy4l6t
From (redirected): https://drive.google.com/uc?export=download&id=1RRV0_qYKGTvD3oCklKfoZQFYqKJy4l6t&confirm=t&uuid=90b7a52d-05fd-4cbe-9851-1834d228f0fb
To: /ictstr01/home/icb/dominik.klein/git_repos/ot_pert_new/competing_methods/combosciplex/chemCPA/combo_sciplex_prep_hvg_filtered.h5ad
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490M/490M [00:13<00:00, 35.6MB/s]


AnnData object with n_obs × n_vars = 63378 × 5000
    obs: 'ncounts', 'well', 'plate', 'cell_line', 'replicate', 'time', 'dose_value', 'pathway1', 'pathway2', 'perturbation', 'target', 'pathway', 'dose_unit', 'celltype', 'disease', 'tissue_type', 'organism', 'perturbation_type', 'ngenes', 'percent_mito', 'percent_ribo', 'nperts', 'chembl-ID', 'condition', 'condition_ID', 'control', 'cell_type', 'smiles_rdkit', 'source', 'sample', 'Size_Factor', 'n.umi', 'RT_well', 'Drug1', 'Drug2', 'Well', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'split', 'condition_old', 'pert_type', 'batch', 'split_1ct_MEC', 'split_2ct_MEC', 'split_3ct_MEC', 'batch_cov', 'batch_cov_cond', 'log_dose', 'cov_drug_dose'
    var: 'ensembl_id-0', 'ncounts-0', 'ncells-0', 'symbol-0', 'symbol-1', 'id-1', 'n_cells-1', 'mt-1', 'n_cells_by_counts-1', 'mean_counts-1', 'pct_dropout_by_counts-1', 'total_counts-1', 'highly_variable-1', 'means-1', 'dispersions-1', 'dispersions_nor

In [6]:
adata.obs["index"] = adata.obs_names.values
adata.obs["index_adapted"] = adata.obs.apply(lambda x: x["index"][:-2], axis=1)
adata.obs.set_index("index_adapted", inplace=True)

In [7]:
adata_combined.obs["cov_drug_dose"] = adata.obs["cov_drug_dose"]
adata_combined.obs["condition_ID"] = adata.obs["condition_ID"]
adata_combined.obs["log_dose"] = adata.obs["log_dose"]
adata_combined.obs["smiles_rdkit"] = adata.obs["smiles_rdkit"]

In [8]:
frac_valid = adata[adata.obs["split_1ct_MEC"] == "valid"].n_obs / (
    adata[adata.obs["split_1ct_MEC"] == "valid"].n_obs + adata[adata.obs["split_1ct_MEC"] == "train"].n_obs
)

In [9]:
import numpy as np


def create_split(x):
    if x["split"] != "train":
        return x["split"]
    is_train = np.random.choice(2, p=[frac_valid, 1 - frac_valid])
    if is_train:
        return "train_train"
    return "train_valid"


adata_combined.obs["split_1ct_MEC"] = adata_combined.obs.apply(create_split, axis=1)

In [10]:
adata_combined.obs["condition_ID"] = adata_combined.obs["condition_ID"].cat.remove_unused_categories()

In [11]:
adata_combined[adata_combined.obs["log_dose"]=="1.0"].obs["Drug2"].value_counts()

Drug2
control    1951
Name: count, dtype: int64

In [12]:
cpa.CPA.setup_anndata(adata_combined,
                      perturbation_key='condition_ID',
                      dosage_key='log_dose',
                      control_group='CHEMBL504',
                      batch_key=None,
                      smiles_key='smiles_rdkit',
                      is_count_data=False,
                      categorical_covariate_keys=['cell_type'],
                      #deg_uns_key='rank_genes_groups_cov',
                      deg_uns_cat_key='cov_drug_dose',
                      max_comb_len=2,
                     )

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 63878/63878 [00:00<00:00, 887285.20it/s]
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


In [13]:
ae_hparams = {'n_latent': 64,
 'recon_loss': 'gauss',
 'doser_type': 'linear',
 'n_hidden_encoder': 256,
 'n_layers_encoder': 3,
 'n_hidden_decoder': 512,
 'n_layers_decoder': 2,
 'use_batch_norm_encoder': True,
 'use_layer_norm_encoder': False,
 'use_batch_norm_decoder': True,
 'use_layer_norm_decoder': False,
 'dropout_rate_encoder': 0.25,
 'dropout_rate_decoder': 0.25,
 'variational': False,
 'seed': 6478}

trainer_params = {'n_epochs_kl_warmup': None,
 'n_epochs_pretrain_ae': 50,
 'n_epochs_adv_warmup': 100,
 'n_epochs_mixup_warmup': 10,
 'mixup_alpha': 0.1,
 'adv_steps': None,
 'n_hidden_adv': 128,
 'n_layers_adv': 3,
 'use_batch_norm_adv': False,
 'use_layer_norm_adv': False,
 'dropout_rate_adv': 0.2,
 'reg_adv': 10.0,
 'pen_adv': 0.1,
 'lr': 0.0003,
 'wd': 4e-07,
 'adv_lr': 0.0003,
 'adv_wd': 4e-07,
 'adv_loss': 'cce',
 'doser_lr': 0.0003,
 'doser_wd': 4e-07,
 'do_clip_grad': False,
 'gradient_clip_value': 1.0,
 'step_size_lr': 10}

In [14]:
model = cpa.CPA(adata=adata_combined,
                split_key='split_1ct_MEC',
                train_split='train_train',
                valid_split='train_valid',
                test_split='ood',
                use_rdkit_embeddings=True,
                **ae_hparams,
               )

(18, 2048)


[rank: 0] Global seed set to 6478


In [15]:
adata_combined.obs["cell_type"] = adata_combined.obs["cell_line"]

In [16]:
model.train(max_epochs=200,
            use_gpu=True,
            batch_size=512,
            plan_kwargs=trainer_params,
            early_stopping_patience=10,
            check_val_every_n_epoch=5,
            save_path="/lustre/groups/ml01/workspace/ot_perturbation/models/chemcpa/combosciplex",
           )

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 96.94it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-PCIE-40GB MIG 3g.20gb') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [MIG-5e9a7bb6-017a-5431-af98-01fe948e1fa7]


Epoch 5/200:   2%|▌                           | 4/200 [01:03<49:36, 15.19s/it, v_num=1, recon=-2.4e+3, r2_mean=0.908, adv_loss=2.43, acc_pert=0.159]


Epoch 00004: cpa_metric reached. Module best state updated.


Epoch 10/200:   4%| | 9/200 [02:20<47:01, 14.77s/it, v_num=1, recon=-2.84e+3, r2_mean=0.917, adv_loss=2.43, acc_pert=0.161, val_recon=-2.63e+3, disn
disnt_basal = 0.10023544475944983
disnt_after = 0.2093423000771
val_r2_mean = 0.9068656471982209
val_r2_var = 0.7022016308709644
Epoch 20/200:  10%| | 19/200 [04:50<44:09, 14.64s/it, v_num=1, recon=-3.1e+3, r2_mean=0.927, adv_loss=2.46, acc_pert=0.169, val_recon=-2.96e+3, disn
disnt_basal = 0.08754760966955556
disnt_after = 0.16194057205328866
val_r2_mean = 0.914450406024753
val_r2_var = 0.7235883323833195
Epoch 30/200:  14%|▏| 29/200 [07:19<41:29, 14.56s/it, v_num=1, recon=-3.28e+3, r2_mean=0.929, adv_loss=2.48, acc_pert=0.167, val_recon=-3.14e+3, dis
disnt_basal = 0.07508656842179848
disnt_after = 0.13501464316608144
val_r2_mean = 0.9236708014340512
val_r2_var = 0.7457544010576576
Epoch 40/200:  20%|▏| 39/200 [09:48<38:59, 14.53s/it, v_num=1, recon=-3.41e+3, r2_mean=0.929, adv_loss=2.54, acc_pert=0.155, val_recon=-3.3e+3, disn
disnt_basa

In [17]:
ctrl_adata = adata_ood[adata_ood.obs['condition'] == 'control'].copy()

In [18]:
adata_combined.layers["X_true"] = adata_combined.X
adata_combined.X = ctrl_adata.X[np.random.choice(ctrl_adata.n_obs, size=adata_combined.n_obs, replace=True), :]

In [19]:
model.predict(adata_combined)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 1997/1997 [01:20<00:00, 24.68it/s]


In [20]:
adata_combined.X = adata_combined.layers["X_true"]

In [21]:
adata_combined.write(
    "/lustre/groups/ml01/workspace/ot_perturbation/models/chemcpa/combosciplex/adata_with_predictions_30.h5ad"
)